In [1]:
import csv
import numpy as np
from sklearn import linear_model
from sklearn import preprocessing

from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, HalvingGridSearchCV

In [2]:
def get_river_dataset(fname, pr_list=None, y_name='H_max'):
    pr_arr = []
    y_arr = []
    with open(fname, newline='') as f:
        reader = csv.DictReader(f, delimiter=';')
        for row in reader:
            pr_arr_row = []
            for pr in pr_list:
                pr_arr_row.append(row[pr])

            pr_arr.append(pr_arr_row)
            y_arr.append(row[y_name])
    X = np.asarray(pr_arr, dtype=np.float64)
    y = np.asarray(y_arr, dtype=np.float64)
    return X, y

#### Сумма, средний, высший, низший уровни

In [3]:
def get_sum(h_max):
    return np.sum(h_max)
    
def get_avg(h_max):
    return np.mean(h_max)
    
def get_max(h_max):
    return np.amax(h_max)
    
def get_min(h_max):
    return np.amin(h_max)

#### Среднеквадратическая погрешность прогноза S

In [4]:
def get_s(h_max, h_forecast=None):
    # Среднеквадратическая погрешность прогноза
    n = h_max.shape[0]
    sqr_diff = np.sum((h_max - h_forecast) ** 2) / (n - 1)
    std = sqr_diff ** 0.5
    return std    

#### Среднеквадратическое отклонение sigma

In [5]:
def get_sigma(h_max):
    # Среднеквадратическая погрешность климатическая.
    # Рассчитывается только по всей совокупности данных.
    return np.std(h_max, ddof=1)

In [6]:
def get_hmax_avg(h_max):
    # Среднее значение h_max.
    # Рассчитывается только по всей совокупности данных.
    return np.mean(h_max)

#### Допустимая погрешность прогноза delta_dop

In [7]:
def get_delta_dop(sigma):
    return 0.674 * sigma

#### Критерий эффективности метода прогнозирования климатический S/sigma

In [8]:
def get_criterion(s, sigma):
    return s / sigma

#### Климатическая обеспеченность Pk

In [9]:
def get_pk(h_max, h_max_avg, delta_dop):
    diff = np.abs(h_max - h_max_avg) / delta_dop
    trusted_values = diff[diff <= 1.0]
    m = trusted_values.shape[0]
    n = h_max.shape[0]
    return m / n * 100.00

#### Обеспеченность метода (оправдываемость) Pm

In [10]:
def get_pm(h_max, h_forecast, delta_dop):
    diff = np.abs(h_max - h_forecast) / delta_dop
    trusted_values = diff[diff <= 1.0]
    m = trusted_values.shape[0]
    n = h_max.shape[0]
    return m / n * 100.00

#### Корреляционное отношение

In [11]:
def get_correlation_ratio(criterion):
    c_1 = (1 - criterion ** 2)
    ro = c_1 ** 0.5 if c_1 > 0 else 0
    return ro

#### Вероятная ошибка прогноза S'

In [12]:
def get_forecast_error(s):
    return 0.674 * s

#### Ошибки климатического/природного прогноза для каждого года delta50

In [13]:
def get_delta50(h_max, delta_dop, h_max_avg=None, h_max_forecast=None):
    if h_max_forecast is None:
        # delta50 климатическая
        return (h_max - h_max_avg) / delta_dop
    else:
        # delta50 прогноза
        return (h_max - h_max_forecast) / delta_dop
  

#### Функция записи в csv файл

In [14]:
import csv
def write_dataset_csv(dataset, dataset_name, fieldnames, pr_group):
    with open(f'results/{dataset_name}/group-{pr_group}/{dataset_name}-гр{pr_group}.csv', 'w', newline='', encoding='utf-8') as csvfile:# , encoding='utf-8'
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter=';', extrasaction='ignore')
        writer.writeheader()
        writer.writerows(dataset)


#### Функция разделения набора данных на тренировочный и тестовый

In [15]:
def train_test_split(X, y, n_test):
    X_train = X[:-n_test]
    y_train = y[:-n_test]
    X_test = X[-n_test:]
    y_test = y[-n_test:]
    return X_train, y_train, X_test, y_test

#### Функция формирования тестового набора данных с подстановкой нормированных значений

In [16]:
def test_norm(x, pr_list, norms):
    x_norm = np.copy(x)
    for col, pr in enumerate(pr_list):
        if pr in norms:
            x_norm[:, col:col+1] = norms[pr]
    return x_norm

#### Функция получения датасетов

In [17]:
def get_datasets():
    datasets = {
        'Неман-Белица': 'Неман',
        # 'Неман-Гродно': 'Неман',
        # 'Неман-Мосты': 'Неман',
        # 'Неман-Столбцы': 'Неман',
    }
    return datasets

#### Функция получения списка предикторов по названию датасета

In [18]:
def get_predictors(dataset_name, pr_group):

    datasets = get_datasets()   
    predictors_lists = {
        'Неман': (
            ['S_2802', 'Smax', 'H_2802', 'X', 'X1', 'X2', 'X3', 'Xs'],
            ['Smax', 'H_2802', 'X', 'X1', 'X3'],
            ['S_2802', 'H_2802', 'X2', 'X3', 'Xs'],
        )
    }
    return predictors_lists[datasets[dataset_name]][pr_group]
    

In [19]:
def get_norms(dataset_name):
    norms_list = {
        'Неман-Белица': {'X1': 46.0, 'X2':35.0},
        'Неман-Гродно': {'X1': 36.0, 'X2':26.0},
        'Неман-Мосты': {'x1': 40.0, 'x2':31.0},
        'Неман-Столбцы': {'x1': 43.0, 'x2':34.0},
    }
    return norms_list[dataset_name]

    

In [20]:
np.logspace(-2, 3, num=100)

array([1.00000000e-02, 1.12332403e-02, 1.26185688e-02, 1.41747416e-02,
       1.59228279e-02, 1.78864953e-02, 2.00923300e-02, 2.25701972e-02,
       2.53536449e-02, 2.84803587e-02, 3.19926714e-02, 3.59381366e-02,
       4.03701726e-02, 4.53487851e-02, 5.09413801e-02, 5.72236766e-02,
       6.42807312e-02, 7.22080902e-02, 8.11130831e-02, 9.11162756e-02,
       1.02353102e-01, 1.14975700e-01, 1.29154967e-01, 1.45082878e-01,
       1.62975083e-01, 1.83073828e-01, 2.05651231e-01, 2.31012970e-01,
       2.59502421e-01, 2.91505306e-01, 3.27454916e-01, 3.67837977e-01,
       4.13201240e-01, 4.64158883e-01, 5.21400829e-01, 5.85702082e-01,
       6.57933225e-01, 7.39072203e-01, 8.30217568e-01, 9.32603347e-01,
       1.04761575e+00, 1.17681195e+00, 1.32194115e+00, 1.48496826e+00,
       1.66810054e+00, 1.87381742e+00, 2.10490414e+00, 2.36448941e+00,
       2.65608778e+00, 2.98364724e+00, 3.35160265e+00, 3.76493581e+00,
       4.22924287e+00, 4.75081016e+00, 5.33669923e+00, 5.99484250e+00,
      

#### Функция обучения и оценки моделей

In [22]:
def compare_models(pr_group, n_test=None, norms=True, top_best=None):
    from sklearn.linear_model import LinearRegression
    from sklearn.linear_model import Ridge
    from sklearn.linear_model import Lasso
    from sklearn.linear_model import ElasticNet
    from sklearn.linear_model import Lars
    from sklearn.linear_model import LassoLars
    from sklearn.linear_model import OrthogonalMatchingPursuit
    from sklearn.linear_model import BayesianRidge
    from sklearn.linear_model import ARDRegression
    from sklearn.linear_model import TweedieRegressor
    from sklearn.linear_model import SGDRegressor
    from sklearn.linear_model import PassiveAggressiveRegressor
    from sklearn.linear_model import HuberRegressor
    from sklearn.linear_model import TheilSenRegressor
    from sklearn.linear_model import QuantileRegressor

    import scipy.stats as stats
    from sklearn.pipeline import Pipeline, make_pipeline
    from sklearn.feature_selection import SelectKBest, SelectFromModel
    from sklearn.feature_selection import r_regression  
    
    ds_dir = 'data'
    
    names = [
        'LinearRegression',
        'Ridge',
        'Lasso',
        'ElasticNet',
        'Lars1',
        'Lars2',
        'Lars3',
        'Lars4',
        'Lars5',
        'LassoLars',
        'OMP1',
        'OMP2',
        'OMP3',
        'OMP4',
        'OMP5',
        'BayesianRidge',
        'ARDRegression',
        'SGDRegressor', 
        'PassiveAggressiveRegressor',
        'HuberRegressor',
        'TheilSenRegressor',
        'QuantileRegressor',               
    ]

    # Инициализация генератора случайных чисел для
    # для обеспечения воспроизводимости результатов
    rng = np.random.RandomState(0)

    # Наборы гиперпараметров моделей для алгоритма кроссвалидации
    # Гиперпараметры для Ridge, Lasso, ElasticNet, LassoLars, HuberRegressor
    alphas = np.logspace(-4, 3, num=100)
    # Гиперпараметры для ElasticNet
    l1_ratio = np.linspace(0.01, 1.0, num=50)
    # Гиперпараметры для BayesianRidge
    alphas_init = np.linspace(0.5, 2, 5)
    lambdas_init = np.logspace(-3, 1, num=5)
    # Гиперпараметры для ARDRegression
    alphas_lambdas = np.logspace(-7, -4, num=4)
    # Гиперпараметры для SGDRegressor
    losses = ['squared_error', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive']
    sgd_alphas = np.logspace(-4, 1, num=100)
    # Гиперпараметры для PassiveAggressiveRegressor
    cc = np.linspace(0.1, 1.5, 50)
    # Гиперпараметры для HuberRegressor
    epsilons = np.append(np.linspace(1.1, 2.0, 10), [1.35])
    # Гиперпараметры для TheilSenRegressor
    n_subsamples = np.arange(6, 24)
    # Гиперпараметры для QuantileRegressor
    q_alphas = np.linspace(0, 2, 20)    
           
    regressors = [
        LinearRegression(),
        GridSearchCV(estimator=Ridge(random_state=rng), param_grid={"alpha": alphas}),
        GridSearchCV(estimator=Lasso(random_state=rng), param_grid={"alpha": alphas}),
        GridSearchCV(estimator=ElasticNet(random_state=rng), param_grid={"alpha": alphas, "l1_ratio": l1_ratio}, n_jobs=-1),
        Lars(n_nonzero_coefs=1),
        Lars(n_nonzero_coefs=2),
        Lars(n_nonzero_coefs=3),
        Lars(n_nonzero_coefs=4),
        Lars(n_nonzero_coefs=5),
        GridSearchCV(estimator=LassoLars(random_state=rng), param_grid={"alpha": alphas}),
        OrthogonalMatchingPursuit(n_nonzero_coefs=1),
        OrthogonalMatchingPursuit(n_nonzero_coefs=2),
        OrthogonalMatchingPursuit(n_nonzero_coefs=3),
        OrthogonalMatchingPursuit(n_nonzero_coefs=4),
        OrthogonalMatchingPursuit(n_nonzero_coefs=5),
        GridSearchCV(estimator=BayesianRidge(), param_grid={"alpha_init": alphas_init, "lambda_init": lambdas_init}, n_jobs=-1),
        GridSearchCV(estimator=ARDRegression(), param_grid={"alpha_1": alphas_lambdas, "alpha_2": alphas_lambdas,"lambda_1": alphas_lambdas,"lambda_2": alphas_lambdas}, n_jobs=-1),
        GridSearchCV(estimator=SGDRegressor(random_state=rng), param_grid={"loss": losses, "alpha": sgd_alphas}, n_jobs=-1),
        GridSearchCV(estimator=PassiveAggressiveRegressor(random_state=rng), param_grid={"C": cc}, n_jobs=-1, cv=3),
        GridSearchCV(estimator=HuberRegressor(), param_grid={"epsilon": epsilons, "alpha": alphas}, n_jobs=-1, cv=5),
        GridSearchCV(estimator=TheilSenRegressor(random_state=rng), param_grid={"n_subsamples": n_subsamples}, n_jobs=-1, cv=5),
        GridSearchCV(estimator=QuantileRegressor(), param_grid={"alpha": q_alphas}, n_jobs=-1),
    ]

    datasets = get_datasets()

    fieldnames = ['Predictors', 'Equations', 'Method', 'Criterion', 'Correlation', 'Pm']

    # Описание структуры данных переменной datasets_result
    # datasets_result = {
    #     "hydropost_0": [
    #         { model_row }
    #         { model_row }
    #     ],
    #     ...,
    #     "hydropost_n": [
    #         { model_row }
    #         { model_row }
    #     ],
    # }
    
    
    # Итерация по датасетам
    datasets_result = dict()
    for ds in datasets:
        result_list = []
        
        pr_list = get_predictors(ds, pr_group)
        
        X, y = get_river_dataset(f'{ds_dir}/{ds}.csv', pr_list=pr_list)

        if n_test is not None and n_test != 0:
            X_train, y_train, X_test, y_test = train_test_split(X, y, n_test)
        else:
            X_train = X[:]
            y_train = y[:]
            X_test = X_train
            y_test = y_train

        if norms:
            norms = get_norms(ds)
            X_test = test_norm(X_test, pr_list, norms)
            
        # Итерация по моделям регрессии
        for name, regr in zip(names, regressors):
            one_model_row = dict()

            regr.fit(X_train, y_train)
            try:
                regr.transorm(X_train, np.reshape(y_train, (y_train.shape[0], 1)))
            except Exception:
                pass
                
            y_predicted = np.ravel(regr.predict(X_test))
            print(y_predicted)            

            try:
                coef = regr.best_estimator_.coef_
                intercept = regr.best_estimator_.intercept_
                
                if isinstance(intercept, np.ndarray):
                    intercept = intercept[0]
                print('cv_intercept', intercept, type(intercept))
            except Exception as error:
                
                coef = regr.coef_
                intercept = regr.intercept_
                
                if isinstance(intercept, np.ndarray):
                    intercept = intercept[0]
                print('rg_intercept', intercept, type(intercept))
                print(error)
                
            # Коэффициенты уравнения (если есть)
            print('COEF', coef)
            coef = np.around(np.ravel(coef), 3)
            intercept = round(intercept, 3)
            try:
                predictors_coef = {f: c for f, c in zip(pr_list, coef) if c != 0.0}
                predictors = ", ".join(predictors_coef.keys())
                print(intercept, predictors_coef.items())
                equation = str(intercept) + ' ' + ' '.join(str(c) + '*' + f for f, c in predictors_coef.items())
                equation = equation.replace(" -", "-")
                equation = equation.replace(" ", " + ")
                equation = equation.replace("-", " - ")
    
                one_model_row['Predictors'] = predictors
                one_model_row['Equations'] = equation
            except Exception as error:
                print(error)
                one_model_row['Predictors'] = ""
                one_model_row['Equations'] = ""

            # Название датасета
            one_model_row['Dataset_name'] = ds

            # Группа предикторов
            one_model_row['Group'] = pr_group
                
            # Название метода
            one_model_row['Method'] = name

            # Расчет показателей качества по методике

            # Сумма, максимум, минимум максимальных уровней
            one_model_row['H_sum'] = get_sum(y)
            one_model_row['H_max'] = get_max(y)
            one_model_row['H_min'] = get_min(y)
            
            # Среднее значение максимального уровня по всей выборке
            h_max_avg = get_hmax_avg(y)
            one_model_row['H_avg'] = h_max_avg
            
            # Среднеквадратическое отклонение
            sigma = get_sigma(y)
            one_model_row['Sigma'] = sigma

            # Допустимая погрешность прогноза
            delta_dop = get_delta_dop(sigma)
            one_model_row['Delta_dop'] = delta_dop

            # Обеспеченность климатическая Pk
            pk = get_pk(y_test, h_max_avg, delta_dop)
            one_model_row['Pk'] = pk

            # Обеспеченность метода (оправдываемость) Pm
            pm = get_pm(y_test, y_predicted, delta_dop)
            one_model_row['Pm'] = pm

            # Среднеквадратическая погрешность прогноза
            s_forecast = get_s(y_test, y_predicted)
            one_model_row['S'] = s_forecast
            
            # Критерий эффективности метода прогнозирования климатический S/sigma
            criterion_forecast = get_criterion(s_forecast, sigma)
            one_model_row['Criterion'] = criterion_forecast

            # Критерий эффективности метода прогнозирования климатический S/sigma в квадрате
            criterion_sqr = get_criterion(s_forecast, sigma) ** 2.0
            one_model_row['Criterion_sqr'] = criterion_sqr
            
            # Корреляционное отношение ro
            correlation_forecast = get_correlation_ratio(criterion_forecast)
            one_model_row['Correlation'] = correlation_forecast
                        
            # Обученная модель
            one_model_row['Model'] = regr

            # Добавление результатов модели в результирующий список по датасету
            result_list.append(one_model_row)

        # Сортировка результатов по каждому датасету
        result_list.sort(key=lambda row: (row['Criterion'], -row['Correlation'], -row['Pm']))

        datasets_result[ds] = result_list

        # Запись в .csv файл
        write_dataset_csv(result_list, ds, fieldnames, pr_group=pr_group)

        for i, rl in enumerate(result_list):
            if top_best is not None:
                if i >= top_best:
                    break
            verify_forecast(ds, rl, i, pr_group=pr_group, n_test=n_test)

    return datasets_result

#### Функция формирования проверочных прогнозов

In [23]:
def verify_forecast(dataset_name, rl, num, pr_group, n_test=None, norms=True):

    ds_dir = 'data'

    pr_list = get_predictors(dataset_name, pr_group)
    pr_list = ['year'] + pr_list
    
    fieldnames = [
        '№', 'Год',
        'Hmax фактический', 'Hф-Hср', '(Hф-Hср)^2', 'δ50% Погрешность климатических прогнозов в долях от допустимой погрешности',
        'Hmax прогнозный', 'Hф-Hп', '(Hф-Hп)^2', 'δ50% Погрешность проверочных прогнозов в долях от допустимой погрешности',
    ]

    X, y = get_river_dataset(f'{ds_dir}/{dataset_name}.csv', pr_list=pr_list, y_name='H_max')

    if n_test is not None and n_test != 0:
        _, _, X_test, y_test = train_test_split(X, y, n_test)
    else:
        X_test = X
        y_test = y

    if norms:
        norms = get_norms(dataset_name)
        X_test = test_norm(X_test, pr_list, norms)

    # Выделение первой колонки (года) из набора предикторов
    years = X_test[:, 0]
    X_test = X_test[:, 1:]
    
    # Forecast
    h_max_forecast = np.ravel(rl['Model'].predict(X_test))
    
    # Hсредний
    h_max_avg = np.mean(y)

    # H - Hсредний
    diff_fact = y_test - h_max_avg

    # (H - Hсредний) в квадрате
    diff_fact_sqr = diff_fact ** 2

    # Погрешность климатических прогнозов в долях от допустимой погрешности
    delta_dop = get_delta_dop(get_sigma(y))
    error_climate = get_delta50(y_test, delta_dop, h_max_avg=h_max_avg)

    # H - Hпрогнозный
    diff_forecast = y_test - h_max_forecast

    # (H - Hпрогнозный) в квадрате
    diff_forecast_sqr = diff_forecast ** 2       

    # Погрешность проверочных прогнозов в долях от допустимой погрешности
    error_forecast = get_delta50(y_test, delta_dop, h_max_forecast=h_max_forecast)

    # Номер по порядку
    rows_num = y_test.shape[0]
    npp = np.arange(1, rows_num + 1, 1)

    # Конкатенация массивов
    att_tuple = (npp, years, y_test, diff_fact, diff_fact_sqr, error_climate, h_max_forecast, diff_forecast, diff_forecast_sqr, error_forecast)
    arr = np.column_stack(att_tuple)
    arr = arr.tolist()

    # Обеспеченность метода (оправдываемость) Pm
    pm = get_pm(y_test, h_max_forecast, delta_dop)
    
    # Запись проверочного прогноза в csv файл
    with open(f'results/{dataset_name}/group-{pr_group}/{dataset_name}-проверочный-гр{pr_group}-{num:0>2}.csv', 'w', newline='', encoding='utf-8') as csvfile: #, encoding='utf-8'
        stat_header = (
            f"Таблица  - Проверочные прогнозы максимумов весеннего половодья\n"
            f"р.{rl['Dataset_name']}\n"
            f"Предикторы:;; {rl['Predictors']}\n"
            f"Уравнение:;; {rl['Equations']}\n"
            f"Модель:;; {rl['Method']}\n\n"
        )
        csvfile.write(stat_header)
       
        writer = csv.writer(csvfile, delimiter=';')
        writer.writerow(fieldnames)
        writer.writerows(arr)
        
        stat_footer = (
            f"Сумма;;{rl['H_sum']}\n"  
            f"Средний;;{rl['H_avg']}\n" 
            f"Высший;;{rl['H_max']}\n"
            f"Низший;;{rl['H_min']}\n\n"
            
            f"σ = ;;{rl['Sigma']};;σ -;среднеквадратическое отклонение (см)\n" 
            f"δдоп =;;{rl['Delta_dop']};;δдоп -;допустимая погрешность прогноза (см)\n" 
            f"Pк =;;{rl['Pk']};;Pк -;климатическая обеспеченность в %\n"
            f"Pм =;;{rl['Pm']};;Pм -;обеспеченность метода в %\n"
            f"S =;;{rl['S']};;;(допустимой погрешности проверочных прогнозов)\n"
            f"S/σ =;;{rl['Criterion']};;S -;среднеквадратическая погрешность (см)\n" 
            f"(S/σ)^2 =;;{rl['Criterion_sqr']};;S/σ -;критерий эффективности метода прогнозирования\n"
            f"ρ =;;{rl['Correlation']};;ρ -;корреляционное отношение\n"
            f";;;;;(оценка эффективности метода прогнозирования)\n"
            f";;;;δ50% -;погрешность (ошибка) прогнозов (см)\n"
        )
        
        csvfile.write(stat_footer) 

#### Запуск процесса обучения моделей, формирования наборов уравнений множественной регрессии и проверочных прогнозов

In [24]:
top_best = None
result1 = compare_models(pr_group=1, n_test=0, norms=True, top_best=top_best)          
result2 = compare_models(pr_group=2, n_test=0, norms=True, top_best=top_best)

[230.57709273 336.29979341 404.41211451 329.62040277 291.18228692
 347.49006106 201.35250779 268.33968844 316.67756636 368.80163277
 285.19084736 276.52326282 258.41700919 273.68197261 209.91025204
 264.42392224 344.73354168 197.79099127 394.00229566 330.70612064
 206.90080302 286.5420064  295.87340215 266.0842359  320.68584359
 289.70699205 247.24861466 211.27134961 354.25936278 268.63990223
 242.72503073 424.79856223 191.8517008  270.988241   282.68342503
 304.29501038 325.91236089]
rg_intercept 67.03603750643762 <class 'numpy.float64'>
'LinearRegression' object has no attribute 'best_estimator_'
COEF [1.33679606 0.13208292 0.51642084 0.50910961 0.30562716]
67.036 dict_items([('Smax', 1.337), ('H_2802', 0.132), ('X', 0.516), ('X1', 0.509), ('X3', 0.306)])
[236.70096856 331.02509363 400.17344344 316.19063327 293.94889353
 340.84591599 216.07418936 261.18054494 308.02540419 361.23305649
 279.19465696 266.02698023 258.39258685 283.1912762  216.91952466
 259.66955565 345.03112447 217.368

F:\ruslan\projects\pkogo\env_ogo\lib\site-packages\sklearn\cross_decomposition\_pls.py:503: FutureWarning: The attribute `coef_` will be transposed in version 1.3 to be consistent with other linear models in scikit-learn. Currently, `coef_` has a shape of (n_features, n_targets) and in the future it will have a shape of (n_targets, n_features).
  warnings.warn(
F:\ruslan\projects\pkogo\env_ogo\lib\site-packages\sklearn\cross_decomposition\_pls.py:503: FutureWarning: The attribute `coef_` will be transposed in version 1.3 to be consistent with other linear models in scikit-learn. Currently, `coef_` has a shape of (n_features, n_targets) and in the future it will have a shape of (n_targets, n_features).
  warnings.warn(
F:\ruslan\projects\pkogo\env_ogo\lib\site-packages\sklearn\cross_decomposition\_pls.py:503: FutureWarning: The attribute `coef_` will be transposed in version 1.3 to be consistent with other linear models in scikit-learn. Currently, `coef_` has a shape of (n_features, n_t